# Add Songs to DB

Use this notebook to initialize the django db with songs that were calculated before.

The song schemas of the old hsd recommender are a bit different to the schema used in the current db, so some adaptation needs to happen.

In [ ]:
# imports
import os
import requests
import json

In [ ]:
# paths
base_path = "/run/media/chr1s/chr1sdrive1/MuInf/"
json_path = f"{base_path}Audio_jsons"
audio_path = f"{base_path}srv/Audio"
album_art_path = f"{base_path}srv/Album_Art"

In [ ]:
# urls
base_url = "http://localhost:8000"
artwork_url = f"{base_url}/song/artwork-file/"
song_file_url = f"{base_url}/song/song-file/"
song_url = f"{base_url}/song/"

In [ ]:
def read_json(name: str) -> dict:
    path = os.path.join(json_path, name)
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data



In [ ]:
json_file_names = os.listdir(json_path)

songs = []

for file_name in json_file_names:
    if file_name.endswith('.json'):
        try:
            songs.append(read_json(file_name))
        except Exception as e:
            print(f"Error reading {file_name}: {e}")
            continue

In [ ]:
def convert_song_to_db_format(song: dict):
    db_song = {
        "title": song['title'],
        "artist": song['artist'],
        "duration_s": song['duration_s'],
        "features": {
            'valence': song['features']['valence'],
            'arousal': song['features']['arousal'],
            'authenticity': song['features']['authenticity'],
            'timeliness': song['features']['timeliness'],
            'complexity': song['features']['complexity'],
            'danceability': song['features']['danceability'],
            'tonal': song['features']['tonal'],
            'voice': song['features']['voice'],
            'bpm': song['features']['bpm']
        },
        "genres": song['features']['genres'],
    }
    db_album = {
        "album": song['album'],
        "artist": song['artist'],
    }
    return db_song, db_album

def get_artwork_path(song: dict) -> str:
    return os.path.join(album_art_path, song['ids']['artwork_id'])

def get_song_path(song: dict) -> str:
    return os.path.join(audio_path, song['ids']['track_id'])

In [ ]:
def upload_song_to_db(song: dict):
    db_song, db_album = convert_song_to_db_format(song)
    artwork_path = get_artwork_path(song)
    song_path = get_song_path(song)

    #print(f"Artwork path: {artwork_path}")
    #print(f"Song path: {song_path}")

    #print(db_album)

    # Upload album artwork
    with open(artwork_path, 'rb') as artwork_file:
        artwork_response = requests.post(
            artwork_url,
            files={'artwork': artwork_file},
            data={'album': db_album['album'], 'artist': db_album['artist']}
        )
    
    #print(f"Artwork upload response: {artwork_response.status_code}, {artwork_response.text}")

    # upload song file
    with open(song_path, 'rb') as song_file:
        song_response = requests.post(
            song_file_url,
            files={'audio_file': song_file}
        )

    #print(f"Song upload response: {song_response.status_code}, {song_response.text}")

    db_song["audio_file_id"] = song_response.json().get("id")
    db_song["artwork_id"] = artwork_response.json().get("id")
    
    #print(db_song)
    
    # Upload song metadata
    song_metadata_response = requests.post(
        song_url,
        json=db_song
    )
    
    print(f"Song metadata upload response: {song_metadata_response.status_code}, {song_metadata_response.text}")

In [ ]:
for song in songs:
    try:
        upload_song_to_db(song)
    except Exception as e:
        print(f"Error uploading song {song['title']}: {e}")
        continue